# Compile a model for the Edge TPU

This notebook offers a convenient way to compile a TensorFlow Lite model for the Edge TPU, in case you don't have a system that's compatible with the [Edge TPU Compiler](https://coral.ai/docs/edgetpu/compiler/) (Debian Linux only).

Simply upload a compatible `.tflite` file to this Colab session, run the code below, and then download the compiled model.

For more details about how to create a model that's compatible with the Edge TPU, see the [documentation at coral.ai](https://coral.ai/docs/edgetpu/models-intro/).


In [ ]:
#Import YOLOv5 for tflite export of model
#Be sure to upload model you wish to export
!git clone https://github.com/achelm15/yolov5.git
%cd yolov5

In [ ]:
#Upload and Unzip dataset for tflite export
%%capture
!unzip LPCVtrain.zip
%cp -r LPCV/train LPCV/valid

In [ ]:
data = open("data.yaml", "w")
data.write("train: ./LPCV/train/images\nval: ./LPCV/valid/images\nnc: 2\nnames: [\'person\', \'sports-ball\']")
data.close()

In [ ]:
!pip install torch==1.9
!pip install torchvision==0.10

In [ ]:
#Test original model accuracy
!python val.py --weights best.pt --img 512 --data data.yaml

In [ ]:
#Export model, be sure to choose desired image size
!python export.py --weights best.pt --img 512 --include tflite --int8 --data data.yaml

In [ ]:
#Test original model accuracy
!python val.py --weights best.tflite --img 512 --data data.yaml

<a href="https://colab.research.google.com/github/google-coral/tutorials/blob/master/compile_for_edgetpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/google-coral/tutorials/blob/master/compile_for_edgetpu.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


## Upload a compatible TF Lite model

To use this script, you need to upload a TensorFlow Lite model that's fully quantized and meets all the [Edge TPU model requirements](https://coral.ai/docs/edgetpu/models-intro/#model-requirements).

With a compatible model in-hand, you can upload it as follows:

1.  Click the **Files** tab (the folder icon) in the left panel. (Do not change directories.)
2.  Click **Upload to session storage** (the file icon). 
3.  Follow your system UI to select and open your `.tflite` file.
    
    When it's uploaded, you should see the file appear in the left panel.
4.  Replace `example.tflite` with your uploaded model's filename:

5.  Now click **Runtime > Run all** in the Colab toolbar.

## Get the Edge TPU Compiler

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

## Compile the model

In [ ]:
%env TFLITE_FILE=best.tflite

In [ ]:
! edgetpu_compiler -s -i "model/tf_detect/Sigmoid;model/tf_detect/transpose;model/tf_detect/Reshape1" -a $TFLITE_FILE

The compiled model uses the same filename but with "_edgetpu" appended at the end.

If the compilation failed, check the **Files** panel on the left for the `.log` file that contains more details. (You might need to click the **Refresh** button to see the new files.)
